# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

## Откройте и изучите файл

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score

# Для задачи классификации
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
# Кросс-валидация
from sklearn.model_selection import GridSearchCV


df = pd.read_csv('/datasets/users_behavior.csv')
df.head()



,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


In [2]:
df['calls'] = df['calls'].astype(int)
df['messages'] = df['messages'].astype(int)
df['minutes'] = np.ceil(df['minutes']).astype(int)
df['mb_used'] = np.ceil(df['mb_used']).astype(int)
df.head(10)

,calls,minutes,messages,mb_used,is_ultra
0,40,312,83,19916,0
1,85,517,56,22697,0
2,77,468,86,21061,0
3,106,746,81,8438,1
4,66,419,1,14503,0
5,58,345,21,15824,0
6,57,432,20,3739,1
7,15,133,6,21912,0
8,7,44,3,2539,1
9,90,666,38,17359,0


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
calls       3214 non-null int64
minutes     3214 non-null int64
messages    3214 non-null int64
mb_used     3214 non-null int64
is_ultra    3214 non-null int64
dtypes: int64(5)
memory usage: 125.7 KB


## Разбейте данные на выборки

Тут определим основные компоненты для нашей модели классификации

In [4]:
# Признаки
features = df.drop(columns=['is_ultra'])

# Целевая переменная (класс пользователей - Ультра или Смарт) («Ультра» — 1, «Смарт» — 0).
target = df['is_ultra']


# Выделим 20% данных для валидационной и тестовой выборки
features_train, features_valid, target_train, target_valid = train_test_split( \
    features, target, test_size=.40, random_state=12345) 

features_valid, features_test, target_valid, target_test = train_test_split( \
    features_valid, target_valid, test_size=.50, random_state=12345) 


In [5]:
datasets = [features_train, features_valid, features_test]
for d in datasets:
    display(d)

,calls,minutes,messages,mb_used
3027,60,432,26,14752
434,33,266,59,17399
1226,52,342,68,15463
1054,42,227,21,13244
1842,30,199,0,8190
...,...,...,...,...
2817,12,87,22,36629
546,65,459,0,15215
382,144,907,0,25003
2177,38,302,37,28915


,calls,minutes,messages,mb_used
1386,92,537,18,20194
3124,40,287,17,17919
1956,81,532,56,17756
2286,67,461,27,16627
3077,22,121,16,9040
...,...,...,...,...
1999,56,399,4,23683
1023,76,602,0,17105
748,81,526,15,18879
1667,10,64,0,2568


,calls,minutes,messages,mb_used
160,61,496,8,10892
2498,80,556,28,28084
1748,87,698,0,8336
1816,41,276,9,10033
1077,60,429,20,29390
...,...,...,...,...
2401,55,447,79,26527
2928,102,743,58,16090
1985,52,350,42,12151
357,39,222,59,17866


# Вывод (шаг 2)
Разделили исходную выборку на три части с соотношением (3:1:1). Для этого использовался метод train_test_split два раза,где сначала разделили исходные данные на train и valid датасеты,  затем valid пополам.

## Исследуйте модели

### Дерево решений (DecisionTreeClassifier)

In [6]:

# Проведём подбор гиперпараметров для модели

# Тут храним результаты 
best_model = None
best_result = 0

# Варьируем в цикле гиперпараметр глубины дерева и считаем метрику accuracy 
for depth in range(1, 10):
    tree_clf = DecisionTreeClassifier(random_state=12345, max_depth = depth)
    tree_clf.fit(features_train, target_train)
    predictions_valid = tree_clf.predict(features_valid)
    result = accuracy_score(target_valid, predictions_valid)
    if result > best_result:
        best_result, best_model = result, tree_clf
    
print('Лучший Accuracy для модели дерева: ', best_result)
print('Подходящий гиперапараметр глубины: ', best_model.max_depth)

Лучший Accuracy для модели дерева:  0.7853810264385692
Подходящий гиперапараметр глубины:  3


### Случайный Лес(RandomForestClassifier)

In [7]:
best_model = None
best_result = 0

# Для модели леса переберём число деревьев
for est in range(1, 100):
    randf_clf = RandomForestClassifier(random_state=12345, n_estimators=est)
    randf_clf.fit(features_train, target_train)
    predictions_valid = randf_clf.predict(features_valid)
    result = accuracy_score(target_valid, predictions_valid)
    if result > best_result:
        best_result, best_model = result, randf_clf
        
print('Лучший Accuracy для модели дерева: ', best_result)
print('Подходящий гиперапараметр глубины: ', best_model.n_estimators)

Лучший Accuracy для модели дерева:  0.7931570762052877
Подходящий гиперапараметр глубины:  51


Попробуем провести кросс-валидацию для модели случайного леса

In [9]:
param_grid = {
        'n_estimators': range(1, 100)
        
}

rnd_forest = RandomForestClassifier(random_state=12345)
grid_search = GridSearchCV(rnd_forest, param_grid, cv=5, scoring='accuracy')
grid_search.fit(features, target)

print("Best CV score: {:.3f}, best CV n_est: {}".format(
    grid_search.best_score_, grid_search.best_estimator_.n_estimators)
) 

Best CV score: 0.804, best CV n_est: 62


### Логистическая регрессия. (LogisticRegression)

In [9]:
log_regression = LogisticRegression(random_state=12345)

log_regression.fit(features_train, target_train)
predictions_valid = log_regression.predict(features_valid)
result = accuracy_score(target_valid, predictions_valid)


print('Лучший Accuracy для модели лог.регрессии: ', result)


Лучший Accuracy для модели лог.регрессии:  0.7573872472783826


/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


## Проверьте модель на тестовой выборке

In [10]:
# Дерево решений 

tree_clf = DecisionTreeClassifier(random_state=12345, max_depth=3)\
    .fit(features_train, target_train)

predictions_test = tree_clf.predict(features_test)
tree_clf_results = accuracy_score(predictions_test, target_test)

print('Accuracy дерева решений (тестовая выборка): ', tree_clf_results)


Accuracy дерева решений (тестовая выборка):  0.7791601866251944


In [11]:
# Случайный лес

randf_clf = RandomForestClassifier(random_state=12345, n_estimators=51)\
    .fit(features_train, target_train)

predictions_test = randf_clf.predict(features_test)
randf_clf_results = accuracy_score(predictions_test, target_test)

print('Accuracy для леса (тестовая выборка): ', randf_clf_results)

Accuracy для леса (тестовая выборка):  0.7853810264385692


In [12]:
log_regression = LogisticRegression(random_state=12345)\
    .fit(features_train, target_train)

predictions_test = log_regression.predict(features_test)
log_regresion_results = accuracy_score(predictions_test, target_test)


print('Accuracy для лог.регресии (тестовая выборка): ', log_regresion_results)

Accuracy для лог.регресии (тестовая выборка):  0.7402799377916018


/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Получается, что линейная модель на тесте немного не дотянула до установленного по заданию порога, но это не значит, что эта модель не нужна. В реальных проектах начинают с простых моделей, потом переходят к сложным. У простых есть преимущество - скорость. У сложных, если все правильно настроено, качество

## (бонус) Проверьте модели на адекватность

Пока что не совсем понимаю, как это осуществляется

# Вывод

В данной работе мы рассмотрели 3 модели классификации в действии, а именно подобрали адекватные гиперпараметры и оценили метрику качетсва, что дало приемлемые результаты для каждой модели на тестовой выборке.
Логистическая регрессия по качеству не дала 75%, но подобралась близко. На кросс-валидации хорошо показал себя случайный лес, дав качество в 0,8 и с гиперпараметром (кол-во деревьев 62). Сочтём дерево решений и случайный лес ,как наилучшие модели для данной задачи- классификации по тарифам. Если хотим добиться лучшего качетсва- смотрим в сторону сл.леса, а если хотим чуть быстрее - то в сторону дерева. 

Комментарий ревьювера\
Проверка на адекватность позволяет понять, хороша ли выбранная нами лучшая модель, или даже нет смысла с ней работать.\
Для проверки на адекватность принято использовать константную модель, которая всегда предсказывает одно и то же\
Например, можно использовать <a href="https://scikit-learn.org/stable/modules/generated/sklearn.dummy.DummyClassifier.html">DummyClassifier</a>\
     Используется так же, как и другие модели sklearn: fit на обучающей выборке, predict на тестовой


Сейчас часто используются не только базовые модели. Серьезный прорыв в показателях метрики может произвести модель бустинга, идея которого в том, чтобы из нескольких слабых алгоритмов сделать один сильный. Советую посмотреть про <a href="https://catboost.ai/docs/concepts/python-quickstart.html">CatBoost.\
И, конечно, советую в следующих спринтах попробовать GridSearchCV, чтобы подбирать параметры

